In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import xarray as xr

import io
import cmdstanpy
import arviz as az
from IPython.display import Image

import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot, row, column
import bokeh.io
import bokeh.plotting
from bokeh import palettes
from bokeh.models import Legend
from scipy import odr


from bokeh.palettes import Spectral6
from bokeh.models import ColorBar, ColumnDataSource
from bokeh.transform import linear_cmap

from IPython.display import Image
from datetime import datetime, timedelta

bokeh.io.output_notebook()
import holoviews as hv
import bebi103
hv.extension('bokeh')

## Determine GEOS-Chem HEMCO Scaling factor and incorporating model intelligence for ethane and propane


In [ ]:
plane_ATOM = pd.read_csv('../../../data/processing/geoschem_plane_sample_ATOM.csv')
plane_HIPPO = pd.read_csv('../../../data/processing/geoschem_plane_sample_HIPPO.csv')

plane_ATOM_n2ofilt_100tropres = pd.read_csv('../../../data/geos_chem_output/plane_ATOM_n2ofilt_100tropres.csv')

gc_atom_synop_n2ofilt_theta_itp_100tropres = pd.read_csv('../../../data/geos_chem_output/gc_atom_synop_n2ofilt_theta_itp_100tropres.csv')


What is the max and min theta for HIPPO during the winter compared to the raw data

In [ ]:
dfplott = plane_HIPPO.loc[plane_HIPPO['campaign_id'] == 1]

In [ ]:
fh = 350
fw = 350
p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')

p.circle(dfplott.lat,dfplott.theta)
p.xaxis.axis_label='lat'
p.yaxis.axis_label='theta'
p.title = 'filtered HIPPO 1'

bokeh.io.show(p)

## HCN filter 


In [ ]:
def pertcile_indx(array,percentile):
    """ Array is a list of gas measurements. 
    Percentile is the maximum percentile the value should 
    be equal to or greater than to return the indices for.
    """
    percval = [st.stats.percentileofscore(array, x, 'weak') for x in array]
    return [i for i, j in enumerate(percval) if j >= percentile]


In [ ]:
pctile_ind = pertcile_indx(plane_ATOM_n2ofilt_100tropres.copy().hcn.to_list(),87)
dfp = plane_ATOM_n2ofilt_100tropres.copy().iloc[pctile_ind]

fh = 400
fw = 400
p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')
p.circle(plane_ATOM_n2ofilt_100tropres.c2h6.values*1e3,
     plane_ATOM_n2ofilt_100tropres.hcn.values*1e3, size=5, color='red')
p.circle(dfp.c2h6.values*1e3,dfp.hcn.values*1e3, size=5, color='black')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "HCN (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
# p.legend.label_text_font_size = '14pt'


# p.legend.location = "bottom_right"
bokeh.io.show(p)


In [ ]:
pctile_ind = pertcile_indx(plane_ATOM_n2ofilt_100tropres.copy().hcn.to_list(),87)
dfp = plane_ATOM_n2ofilt_100tropres.copy().iloc[pctile_ind]



In [ ]:
plane_ATOM_filt_hcnfilt = plane_ATOM_n2ofilt_100tropres.copy()
plane_ATOM_filt_hcnfilt.loc[pctile_ind,['c2h6','hcn','c3h8']] = np.nan

Check that it worked

In [ ]:
plane_ATOM_filt_hcnfilt['c2h6'][pctile_ind].values

In [ ]:
res = plane_ATOM_filt_hcnfilt[['c2h6','hcn','c3h8']].apply(lambda x: x.fillna(method='backfill'))

In [ ]:
plane_ATOM_filt_hcnfilt['c2h6'] = res['c2h6']
plane_ATOM_filt_hcnfilt['hcn'] = res['hcn']
plane_ATOM_filt_hcnfilt['c3h8'] = res['c3h8']

Get the slope of the wintertime for c3h8 vs c2h6 and c3h8 vs ch4 

In [ ]:
atom_winter = plane_ATOM_filt_hcnfilt.loc[plane_ATOM_filt_hcnfilt['campaign_id'] == 2]

In [ ]:
data = odr.Data(atom_winter.c2h6, atom_winter.c3h8)
odr_obj = odr.ODR(data, odr.unilinear)
output = odr_obj.run()
x_fit = np.linspace(min(atom_winter.c2h6), max(atom_winter.c2h6), 1000)
y_fit = output.beta[0]*x_fit + output.beta[1]



In [ ]:
p = figure(title='winter atom all curtains', width=400, height=400)
p.xaxis.axis_label = 'c2h6'
p.yaxis.axis_label = 'c3h8'
p.circle(atom_winter.c2h6, atom_winter.c3h8, color='red', size=5, line_alpha=0)
p.line(x_fit, y_fit, line_width=2)

bokeh.io.show(p)

In [ ]:
# This is the slope 
output.beta[0]

#### Hierchical Bayesian Model

We can reasonably approximate C2H6 and C3H8 measured by the aircraft/GEOS-Chem to be Lognormally distributed with an approximate error. Lognormal distributions have longer tails, which is appropriate given the outliers we see in the measurements. We can "model" the GEOS-Chem simulated results as follows: 

\begin{align}
&&\mathrm{gcs}_i &\sim \text{LogNorm}(\beta \cdot \mathrm{a}_i, \sigma)\\
&&\sigma &\sim \text{Prior}\\
&&\beta &\sim \text{Prior}\\
\end{align}

Where "gc_i" represents GEOS-Chem simulated C3H8 and C2H6, "aircraft_i" represents measured C3H8 or C2H6 measured by the aircraft. Here, $\beta$ is the scalar we are interested in, that estimates missing HEMCO emissions, and $\sigma$ is the approximate error in GEOS-Chem simulations. 

We "repeated" this experiment with several GEOS-Chem replicates over several days before and after the aircraft path, as explained above. If we were to pool all the data together, each experiment would be governed by identical parameters. However, we know that each replicate is subject to differences mainly due to meterology, and we conclude that the parameters in each replicate experiment should be independent from one another, such that we have k separate models to fit, each looking like the one above.  


We can consider a model in which there is a "master" scaling parameter, which we now call $\beta$, and the values of the scaling parameters of the replicates, which we now call $\beta_1$, may vary from this $\beta$ according to some probability distribution, $g(\beta_1{_i} | \beta)$. So now, we have parameters $\beta_1{_{,1}}$, $\beta_1{_{,2}}$, ... $\beta_1{_{,i}}$ and $\beta$.  The posterior can be written using Bayes's theorem, defining $\beta_1 = (\beta_1{_{,1}}, \beta_1{_{,2}}, ...)$,

\begin{align*}
g(\beta, \beta_1 | \mathrm{a}, \mathrm{gcs}) = \frac{f(\mathrm{a}, \mathrm{gcs} | \beta, \beta_1)g(\beta, \beta_1)}{f(\mathrm{a}, \mathrm{gcs})}\\
\end{align*}

Note though, that the observed values of gc do not directly depend on $\beta$, only on $\beta_1$ and as such, the observations are only indirectly dependent on $\beta$. So, we can write: 



\begin{align*}
g(\beta, \beta_1 | \mathrm{a}, \mathrm{gcs}) = \frac{f(\mathrm{a}, \mathrm{gcs} | \beta_1)g(\beta, \beta_1)}{f(\mathrm{a}, \mathrm{gcs})}\\
\end{align*}

Next, we can rewrite the prior using the definition of conditional probability:

\begin{align*}
g(\beta, \beta_1) = g(\beta_1 | \beta)g(\beta)\\
\end{align*}

Substituting this back into the previous expression for the posterior, we have: 

\begin{align*}
g(\beta, \beta_1 | \mathrm{a}, \mathrm{gcs}) = \frac{f(\mathrm{a}, \mathrm{gcs} | \beta_1)g(\beta_1 | \beta)g(\beta)}{f(\mathrm{a}, \mathrm{gcs})}\\
\end{align*}



In the numerator, we see a chain of dependencies. The gc simulations depend on $\beta_1$. Parameters $\beta_1$ depend on hyperparameter $\beta$. Hyperparameter $\beta$ then has some hyperprior distribution. So, the hierarchical model captures both the experiment-to-experiment variability, as well as the master parameter. 

We have to specify a hyperprior, and a conditional prior, $g(\beta_1 | \beta)$. Here, we have no reason to believe that we can distinguish any one $\beta_1{_i}$ from another prior to the experiment. As such, we can assume the conditional prior to behave in an exchangeable manner, where the label "i" is not dependent on the permutations of the indices. So, our expression for the posterior is: 

\begin{align}
g(\beta_1, \beta \mid \mathrm{a}, \mathrm{gcs}) = \frac{f(\mathrm{a}, \mathrm{gcs}\mid \beta_1)\,\left(
\prod_{i=1}^kg(\beta_{1,i}\mid \beta)\right)\,g(\beta)}{f(\mathrm{a}, \mathrm{gcs})}
\end{align}

We assume that all replicates have the same scale parameter, $\tau$, but differing location parameters. Our statistical model is then defined as follows, with a weakly informative prior on $\tau$. 



\begin{align}
&\tau_{i} = 0.05{\left|\Delta t\right|}_i + 0.01 \\
&\beta \sim \text{Norm}(0.7, 0.2)\\
&\beta_{1,i} \sim \text{Norm}(\beta, \tau_{i})\\
&\alpha = 1/\beta\\
&\sigma_{i} = 0.14\cdot \mathrm{tropht}_i + 0.8\\
&\mathrm{gcs}_i \sim \text{LogNorm}(\beta_{1,i} \cdot \mathrm{a}_i, \sigma_i)\\
\end{align}

\begin{align}
&&\mathrm{a} & = \mathrm{gcs}\cdot \alpha\\
\end{align}

\begin{align*}
g(\alpha) = \prod_{s=1}^k g(\alpha{_s},\alpha{_{1,s}} \mid a_s,gc_s)\\
\end{align*}

For campaign number i, and measurement j:

\begin{align}
&\tau_{ij} = 0.05{\left|\Delta t\right|}_{ij} + 0.01 \\
&\beta_{i} \sim \text{Norm}(0.7, 0.2)\\
&\beta_{1,ij} \sim \text{Norm}(\beta_{i}, \tau_{ij})\\
&\alpha_{i} = 1/\beta_{i}\\
&\sigma_{ij} = 0.14\cdot \mathrm{tropht}_{ij} + 0.8\\
&\mathrm{gcs}_{ij} \sim \text{LogNorm}(\beta_{1,ij} \cdot \mathrm{a}_{ij}, \sigma_{ij})\\
\end{align}

These are functions for tau 

In [ ]:
def get_delta_t(synlabel, plane_path_day_num):
    """ Takes in the synoptic_day_number variable from your gc output.
    This datatype should be an integer. Doesn't take in an array - does it one by one.
    The plane path day number is the number of the synoptic replicates that the plane lands on.
    Ie 2.
    """
    return synlabel - plane_path_day_num


In [ ]:
test = [0,1,2,3,4]
[get_delta_t(r,2) for r in test]

In [ ]:
def tao_mean(a,k,delta_t):
    """ function that returns tau, which is a function of time and 
    the prior for your bayes. This won't go into stan, 
    I just want to get an idea of how it works so I can refine what the slope should be.
    a stands for the slope, and k stands for the vertical shift (tau_0)
    
    delta_t should be a numpy array and a and k should be scalars
    
    
    **** This function was written so that it could either get tau final, or the new mean for 
    a distribution of tau in stan. If you input k = 0, then you are doing the latter.
    """

#     return [(a*abs(dt) + k) for dt in delta_t]
    return a*abs(delta_t) + k


Remember, the default is +/-5 days. When you actually do the fits, you limit to +/-2 days. 

If you are confused you can plot this to see tao for all datapoints.  

In [ ]:
def replace_syn_day_label(synday):
    
    if synday == 0:
        newday = -5
    elif synday == 1:
        newday = -4
    elif synday == 2:
        newday = -3
    elif synday == 3:
        newday = -2
    elif synday == 4:
        newday = -1
    elif synday == 5:
        newday = 0
    elif synday == 6:
        newday = 1
    elif synday == 7:
        newday = 2
    elif synday == 8:
        newday = 3
    elif synday == 9:
        newday = 4
    elif synday == 10:
        newday = 5
        
    return newday

This looks exactly as you would expected to given the number of default synoptic days. 

In [ ]:
gc_atom_synop_n2ofilt_theta_itp_100tropres['tau_dist_means'] = tao_mean(.05,.05,get_delta_t(gc_atom_synop_n2ofilt_theta_itp_100tropres['synoptic_day_number'].values, 5))
gc_atom_synop_n2ofilt_theta_itp_100tropres.head()

Great! Before move on, here are the other priors just as a reminder. 

In [ ]:
fh = 300
fw = 300
p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')
p.line(np.linspace(0, 2), st.norm.pdf(np.linspace(0,2), .7, .2), line_width=2, color='black')
p.xaxis.axis_label = "β"
p.yaxis.axis_label = "PDF"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
bokeh.io.show(p)

In [ ]:
print(st.norm.ppf(0.01, .7, .2))
print(st.norm.ppf(0.99, .7, .2))
print(st.norm.ppf(0.5, .7, .2))

In [ ]:
p1 = hv.Points(
    data = gc_atom_synop_n2ofilt_theta_itp_100tropres,
    kdims=['theta','c3h8']
).opts(padding=.1, title='geos chem')

p2 = hv.Points(
    data = gc_atom_synop_n2ofilt_theta_itp_100tropres,
    kdims=['theta','c2h6']
).opts(padding=.1)

p1+p2

In [ ]:
p1 = hv.Points(
    data = plane_ATOM_n2ofilt_100tropres,
    kdims=['theta','c3h8']
).opts(padding=.1, title='plane')

p2 = hv.Points(
    data = plane_ATOM_n2ofilt_100tropres,
    kdims=['theta','c2h6']
).opts(padding=.1)

p1+p2

So geos chem propane ranges from 5e-6 to 1.4e-3. (ethane ranged higher from 2e-4 to 2.5e-3). This will help inform your sigma for your lognormal. The mu should be around 2e-4 (ethane was 7e-4 : 1e-3), but this is going to be determined by beta*plane in the actual model, but just for trying things, use a median between that range for mu.

In [ ]:
print(st.lognorm.ppf(0.01, 3.5, scale=np.exp(.0002)), "e-4")
print(st.lognorm.ppf(0.99, 3.5, scale=np.exp(.0002)), "e-4")
print(st.lognorm.ppf(0.01, 3.5, scale=np.exp(.001)), "e-3")
print(st.lognorm.ppf(0.99, 3.5, scale=np.exp(.001)), "e-3")

In [ ]:
def sigma_generator_function2(val, tropht_max, tropht_min, base_sigma, sigma_interval_increase): 
    """    
    By plotting the tropht of theta bins, I saw that the largest tropht 
    was about 17km, and the smallest is about 6km. I created 5 intervals to assign a sigma to, which 
    is about every 2km.
    
    6km
    8km
    10km
    12km
    14km
    17km
    
    Each interval is between those vals (like between 6 and 8) etc.
    """
    interval_increase_kilometers = (tropht_max - tropht_min)/5
    
    if (tropht_min + interval_increase_kilometers) > val >= tropht_min:
        return base_sigma + sigma_interval_increase
    
    elif (tropht_min + interval_increase_kilometers*2) > val > (tropht_min + interval_increase_kilometers):
        return base_sigma + sigma_interval_increase*2
    
    elif (tropht_min + interval_increase_kilometers*3) > val > (tropht_min + interval_increase_kilometers*2):
        return base_sigma + sigma_interval_increase*3
    
    elif (tropht_min + interval_increase_kilometers*4) > val > (tropht_min + interval_increase_kilometers*3):
        return base_sigma + sigma_interval_increase*4
    
    elif val > (tropht_min + interval_increase_kilometers*4):
        return base_sigma + sigma_interval_increase*5  

    

In [ ]:
i = 2
j = -1

gc_atom_synop_itp_df = gc_atom_synop_n2ofilt_theta_itp_100tropres.loc[(gc_atom_synop_n2ofilt_theta_itp_100tropres['campaign_id'] == i) & 
                      (gc_atom_synop_n2ofilt_theta_itp_100tropres['flight_lat_gradient'] == j) & 
                      (gc_atom_synop_n2ofilt_theta_itp_100tropres['synoptic_day_number'] <= 7) & 
                        (gc_atom_synop_n2ofilt_theta_itp_100tropres['synoptic_day_number'] >= 3)].copy()

plane_df = plane_ATOM_filt_hcnfilt.loc[(plane_ATOM_filt_hcnfilt['campaign_id'] == i) & 
                   (plane_ATOM_filt_hcnfilt['flight_lat_gradient'] == j)].copy()


# get sigma for likelihood using tropht
max_tropht = np.max(plane_df.tropht.values)
min_tropht = np.min(plane_df.tropht.values)
plane_df['sigma_likelihood'] = np.array([sigma_generator_function2(x,max_tropht,min_tropht,.8,.14) for x in plane_df.tropht.values])



In [ ]:
fh = 300
fw = 300
p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')
p.circle(plane_df.tropht.values, plane_df['sigma_likelihood'].values, size=4, color='black')
p.xaxis.axis_label = "Tropopause height (km)"
p.yaxis.axis_label = "σ"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
bokeh.io.show(p)

## Load models

In [ ]:
bebi103.stan.clean_cmdstan()

In [ ]:
sm = cmdstanpy.CmdStanModel(stan_file="stan_model.stan")

In [ ]:
print(sm.code())

Remember that if you get nans here, it is because you are taking the log of a negative number. Here, it looks good. If you end up getting negatives, you can adjust beta prior and shrink the variance so you don't get a negative tail (move mean over to the right and also decrease the variance). 

Here are the references you used to get the epsilon 
 https://mc-stan.org/docs/2_26/stan-users-guide/truncated-data-section.html and https://mc-stan.org/docs/2_26/reference-manual/program-block-transformed-parameters.html 

### Posterior Predictive

In [ ]:
def plot_with_error_bars(means, confs, names, **kwargs):
    """Make a horizontal plot of means/conf ints with error bars."""
    frame_height = kwargs.pop("frame_height", 150)
    frame_width = kwargs.pop("frame_width", 450)

    p = bokeh.plotting.figure(
        y_range=names, frame_height=frame_height, frame_width=frame_width, **kwargs
    )

    p.circle(x=means, y=names)
    for conf, name in zip(confs, names):
        p.line(x=conf, y=[name, name], line_width=2)

    return p

In [ ]:
gc_atom_synop_itp = gc_atom_synop_n2ofilt_theta_itp_100tropres.copy()
plane_ATOM_n2ofilt = plane_ATOM_filt_hcnfilt.copy()


c_name = []
c_id = []
c_flg = []
species = []
alpha_lower = []
alpha_median = []
alpha_upper = []
alpha_path = []
ppc_l = []
alpha_samples = []


for i in plane_ATOM_n2ofilt['campaign_id'].unique():
    for j in plane_ATOM_n2ofilt.loc[plane_ATOM_n2ofilt['campaign_id'] == i].flight_lat_gradient.unique():
        with bebi103.stan.disable_logging():
        
            # the following is copied from what you need for SBC, 
            # so passing gc_sim data (but doesn't actually need it)

            gc_atom_synop_itp_df = gc_atom_synop_n2ofilt_theta_itp_100tropres.loc[(gc_atom_synop_n2ofilt_theta_itp_100tropres['campaign_id'] == i) & 
                                  (gc_atom_synop_n2ofilt_theta_itp_100tropres['flight_lat_gradient'] == j) & 
                                  (gc_atom_synop_n2ofilt_theta_itp_100tropres['synoptic_day_number'] <= 7) & 
                                    (gc_atom_synop_n2ofilt_theta_itp_100tropres['synoptic_day_number'] >= 3)].copy()

            plane_df = plane_ATOM_filt_hcnfilt.loc[(plane_ATOM_filt_hcnfilt['campaign_id'] == i) & 
                               (plane_ATOM_filt_hcnfilt['flight_lat_gradient'] == j)].copy()


            # get sigma for likelihood using tropht
            max_tropht = np.max(plane_df.tropht.values)
            min_tropht = np.min(plane_df.tropht.values)
            plane_df['sigma_likelihood'] = np.array([sigma_generator_function2(x,max_tropht,min_tropht,.8,.14) for x in plane_df.tropht.values])

            tau_means = tao_mean(.05,.05,get_delta_t(gc_atom_synop_itp_df['synoptic_day_number'].values, 5))

            # get replicates of plane gas measurements for input to hierarchical model
            # and add it to the gc df
            plane_df_sorted = plane_df.sort_values(by=['theta'])
            plane_c3h8 = [[x for x in plane_df_sorted.c3h8.values] for i in gc_atom_synop_itp_df['synoptic_day_number'].unique()]
            plane_c3h8_flat = [item for sublist in plane_c3h8 for item in sublist]        
            gc_atom_synop_itp_df2 = gc_atom_synop_itp_df.copy()
            gc_atom_synop_itp_df2['plane_c3h8'] = plane_c3h8_flat
            gc_atom_synop_itp_df2['tau_means'] = tau_means

            # get replicates of plane sigma for likelihood estimates for input to 
            # the hierarchical model and add it to the gc df
            plane_df_sorted = plane_df.sort_values(by=['theta'])
            plane_sigma = [[x for x in plane_df_sorted.sigma_likelihood.values] for i in gc_atom_synop_itp_df['synoptic_day_number'].unique()]
            plane_sigma_flat = [item for sublist in plane_sigma for item in sublist]        
            gc_atom_synop_itp_df2['plane_sigma'] = plane_sigma_flat


            # get data onto same dataframe for input to model
            df = gc_atom_synop_itp_df2.copy().reset_index()
            tmp_dict, df = bebi103.stan.df_to_datadict_hier(
            df, level_cols=["synoptic_day_number"], data_cols=["theta","c3h8","plane_c3h8","plane_sigma","tau_means"]

            )
            # fix the indexing output
            data = {'N':tmp_dict['N'], 'J_1':tmp_dict['J_1'], 'index_1':tmp_dict['index_1'], 
                        'theta':df.sort_index().theta.values, 'gc_sim':df.sort_index().c3h8.values, 
                        'plane':df.sort_index().plane_c3h8.values, 'sigma':df.sort_index().plane_sigma.values,
                        'tau_':df.sort_index().tau_means.values}
            # fix naming 
            # data['gc_sim'] = data.pop('c2h6')
            # data['plane'] = data.pop('plane_c2h6')
            data_prior_pred = data.copy()
            data_prior_pred.update({   
              "mu_beta":.7,
              "sigma_beta":.2
            })

            # run model 
            samples = sm.sample(data=data_prior_pred, iter_warmup = 2000, iter_sampling=1000, adapt_delta = 0.99, chains=4)
            samples = az.from_cmdstanpy(posterior=samples, posterior_predictive=["gc_sim_ppc"])  

            # ppc
            gc_sim_ppc = samples.posterior_predictive["gc_sim_ppc"].stack({"sample": ("chain", "draw")})
            gc_sim_ppc = gc_sim_ppc.transpose('sample', 'gc_sim_ppc_dim_0')

            alpha_samples.append(samples.posterior.alpha.values.flatten())
            CI_alpha = np.percentile(samples.posterior.alpha.values.flatten(), [2.5, 50, 97.5])

            # save out data 
            if j == -1:
                curtain_name = 'pacific'
            elif j == 1:
                curtain_name = 'atlantic'
            c_name.append('ATom')
            c_id.append(i)
            c_flg.append(curtain_name)
            species.append('c3h8')
            alpha_lower.append(CI_alpha[0])
            alpha_median.append(CI_alpha[1])
            alpha_upper.append(CI_alpha[2])
            

            # alpha_1 pertaining to the actual plane flight 
    #         alpha_path.append(np.percentile(samples.posterior.alpha_1[:,:,2].values.flatten(), [10, 50, 90])[1])
            alpha_path.append(np.mean(samples.posterior.alpha_1[:,:,2].values.flatten()))
            ppc_l.append(gc_sim_ppc)

            # print model diagnostics for each run
            print(str('ATom' + ' ' + str(i) + ' ' + curtain_name))
            bebi103.stan.check_all_diagnostics(samples)

atom_hierarch_scaling = pd.DataFrame()
atom_hierarch_scaling['campaign_name'] = c_name
atom_hierarch_scaling['campaign_id'] = c_id
atom_hierarch_scaling['curtain'] = c_flg
atom_hierarch_scaling['species'] = species
atom_hierarch_scaling['alpha_lower'] = alpha_lower
atom_hierarch_scaling['alpha_median'] = alpha_median
atom_hierarch_scaling['alpha_upper'] = alpha_upper
atom_hierarch_scaling['alpha_path'] = alpha_path
atom_ppc = ppc_l

In [ ]:
counter = [0,1,2,3,4,5,6,7]
df_alpha_only = pd.DataFrame()
df_alpha_only['campaign_name'] = ['atom' for curtain in alpha_samples for val in curtain]
df_alpha_only['campaign_id'] = [atom_hierarch_scaling['campaign_id'].values[c] for c in counter for campaign in alpha_samples[c]]
df_alpha_only['curtain'] = [atom_hierarch_scaling['curtain'].values[c] for c in counter for campaign in alpha_samples[c]]
df_alpha_only['species'] = ['C3H8' for curtain in alpha_samples for val in curtain] 
df_alpha_only['posterior'] = [val for curtain in alpha_samples for val in curtain]
df_alpha_only.head()

In [ ]:
df_alpha_only.to_csv('./' + 'c3h8_alpha_hyperparam' + '.csv', index=False)

In [ ]:
df_mcmc = bebi103.stan.arviz_to_dataframe(samples)

In [ ]:
df_mcmc.head()

In [ ]:
fh = 400
fw = 500
title_name = 'Posterior'
color = ["#e78ac3","#8da0cb","black","#fc8d62","#66c2a5"]
p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title=title_name)
legend_it=[]
for i in range(0,5):
    if i == 0:
        synlabel = "plane path - 2 days"
    elif i == 1:
        synlabel = "plane path - 1 days"
    elif i == 2:
        synlabel = "plane path"
    elif i == 3:
        synlabel = "plane path + 1 days"
    elif i == 4:
        synlabel = "plane path + 2 days"
        
    param_name = 'alpha_1[' + str(i) + ']'
    c=p.circle(x=df_mcmc['alpha'].values, y=df_mcmc[param_name].values, size=2, fill_alpha=.3, 
             color=color[i])
#     p.circle(x=df_mcmc['alpha'].values, y=df_mcmc[param_name].values, size=2, fill_alpha=.3, 
#          color=color[i],legend_label=synlabel)
    legend_it.append((synlabel, [c]))


legend = Legend(items=legend_it)
p.add_layout(legend, 'right')
p.xaxis.axis_label = "𝛼"
p.yaxis.axis_label = "𝛼₁"
p.legend.label_text_font_size = '14pt'
# p.legend.location = "bottom_right"
p.xaxis.axis_label_text_font_size = "20pt"
p.yaxis.axis_label_text_font_size = "20pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'


bokeh.io.show(p)

In [ ]:
names = []
confs = []
for i in range(len(atom_hierarch_scaling.campaign_name)):
#     names.append(str(atom_hierarch_scaling.campaign_name.values[i]) + ' ' +
#         str(atom_hierarch_scaling.campaign_id.values[i]) + ' ' +
#         str(atom_hierarch_scaling.curtain.values[i]))
    confs.append([atom_hierarch_scaling.alpha_lower.values[i], atom_hierarch_scaling.alpha_upper.values[i]])

# actually, instead of the actual names, use the season for easier comparison
names = ['Summer Pacific 2016','Summer Atlantic 2016','Winter Pacific 2017','Winter Atlantic 2017',
         'Fall Pacific 2017','Fall Atlantic 2017','Spring Pacific 2018', 'Spring Atlantic 2018']
means = atom_hierarch_scaling.alpha_path.values

In [ ]:
# bokeh.io.show(plot_with_error_bars(means, confs, names, x_axis_label='alpha path'))

In [ ]:
# note that you used the mean for alpha_path above, 
# and here you are reporting the median

medians = atom_hierarch_scaling.alpha_median.values
p = plot_with_error_bars(medians, confs, names, x_axis_label='alpha hyperparam')
p.xaxis.axis_label = "𝛼"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "12pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
bokeh.io.show(p)

alpha path is the alpha estimated during the actual plane path time (synoptic replicate #2). alpha hyperparam is the alpha estimated as a result of all the hierarchy synoptic replicates within each campaign.

## Simulation - based calibration
- Can our model and sampling procedure capture the real parameter values of the generative process?
- Can measured data inform the parameters?
- Can our sampling technique handle the model and any conceivable data set we can throw at it?

1. Draw a parameter set 𝜃̃  out of the prior.
2. Use 𝜃̃  to draw a data set 𝑦̃  out of the likelihood.
3. Perform MCMC sampling of the posterior using 𝑦̃  as if it were the actual measured data set. Draw 𝐿 MCMC samples of the parameters.
4. Do steps 1-3 𝑁 times (hundreds to a thousand is usually a good number).

In step 3, we are using a data set for which we know the underlying parameters that generated it. Because the data were generated using 𝜃̃  as the parameter set, 𝜃̃  is now the ground truth parameter set. So, we can check to see if we uncover the ground truth in the posterior sampling. We can also check diagnostics for each of the trials to make sure the sampler works properly. Furthermore, we can see if the posterior is narrower than the prior, meaning that the data are informing the model. 

In [ ]:
name_of_sbc_csv = 'df_sbc_hier_atom2_pacific_032121'

##### I just ran the following below, which means that I used Atom 4 atlantic for this SBC

In [ ]:
cores=2
df_sbc = bebi103.stan.sbc(
    prior_predictive_model=sm_prior_pred,
    posterior_model=sm,
    prior_predictive_model_data=data_prior_pred,
    posterior_model_data=data_prior_pred,
    measured_data=["gc_sim"],
    var_names=["beta_", "beta_1"],
    measured_data_dtypes=dict(gc_sim=float),
    posterior_predictive_var_names=["gc_sim_ppc"],
    sampling_kwargs=dict(iter_warmup = 2000, iter_sampling=1000, adapt_delta = 0.99), 
    cores=cores,
    N=400,
    progress_bar=True,
)
#sampling_kwargs=dict(warmup_iters=2000, sampling_iters=2000),
#output_dir='./'
df_sbc.to_csv('./' + name_of_sbc_csv + '.csv',  index=False)

In [ ]:
df_sbcr = pd.read_csv('df_sbc_hier_atom2_pacific_032121.csv')

In [ ]:
fh = 400
fw = 500

color = ["black","#e78ac3","#8da0cb","#fc8d62","#66c2a5",'#a55194']
p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')
legend_it=[]
for i in range(0,6):
    if i == 0:
        data = df_sbcr.loc[df_sbcr['parameter'] == 'beta_']
        synlabel = "β"
        
    else:
        data = df_sbcr.loc[df_sbcr['parameter'] == 'beta_1['+str(i-1)+']']
        if i == 1:
            synlabel = "β₁, plane path - 2 days"
        elif i == 2:
            synlabel = "β₁, plane path - 1 days"
        elif i == 3:
            synlabel = "β₁, plane path"
        elif i == 4:
            synlabel = "β₁, plane path + 1 days"
        elif i == 5:
            synlabel = "β₁, plane path + 2 days"
        
   
    c=p.circle(x=data['shrinkage'].values, y=data['z_score'].values, size=5, fill_alpha=.3, 
             color=color[i])
    legend_it.append((synlabel, [c]))


legend = Legend(items=legend_it)
p.add_layout(legend, 'right')
p.xaxis.axis_label = "shrinkage"
p.yaxis.axis_label = "z-score"
p.legend.label_text_font_size = '14pt'
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'



bokeh.io.show(p)